# Hyperparameter Tuning

In this project we will test different approaches to hyperparameter tuning.

We will use some of the models we've created in the [Text Classification with Neural Networks](https://github.com/j-n-t/natural_language_processing/blob/master/Text%20Classification%20with%20Neural%20Networks.ipynb) notebook and will try to improve their performance.

In order to do that, we will test the following methods:

* **Grid Search**
* **Random Search**

### Step 1

#### 1. Perform initial imports

In [1]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Embedding
from keras.layers import SpatialDropout1D, Conv1D, GlobalMaxPooling1D
from keras.layers import LSTM
from keras.layers.wrappers import Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV

import os

import numpy as np
import pandas as pd

Using TensorFlow backend.


#### 2. Load data

In [2]:
# create function to load data

def load_data(n_unique_words):
    (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=n_unique_words)
    return (x_train, y_train), (x_test, y_test)

In [3]:
n_unique_words = 10000 #number of most frequent words to consider

(x_train, y_train), (x_test, y_test) = load_data(n_unique_words)

In [4]:
len(x_train[0])

218

#### 3. Preprocess data

In [5]:
# create function to preprocess data

def preprocess_data(x_train, x_test, max_review_length, pad_type, trunc_type):
    x_train = pad_sequences(x_train, maxlen=max_review_length, 
                            padding=pad_type, truncating=trunc_type, value=0)

    x_test = pad_sequences(x_test, maxlen=max_review_length, 
                           padding=pad_type, truncating=trunc_type, value=0)
    
    return x_train, x_test

In [6]:
max_review_length = 100 #maximum review length of 100 words
pad_type = 'pre' #add padding characters to the start of every review < 100 words
trunc_type = 'pre' #remove words from the beginning of every review > 100 words

x_train, x_test = preprocess_data(x_train, x_test, max_review_length, pad_type, trunc_type)

In [7]:
x_train[0]

array([1415,   33,    6,   22,   12,  215,   28,   77,   52,    5,   14,
        407,   16,   82,    2,    8,    4,  107,  117, 5952,   15,  256,
          4,    2,    7, 3766,    5,  723,   36,   71,   43,  530,  476,
         26,  400,  317,   46,    7,    4,    2, 1029,   13,  104,   88,
          4,  381,   15,  297,   98,   32, 2071,   56,   26,  141,    6,
        194, 7486,   18,    4,  226,   22,   21,  134,  476,   26,  480,
          5,  144,   30, 5535,   18,   51,   36,   28,  224,   92,   25,
        104,    4,  226,   65,   16,   38, 1334,   88,   12,   16,  283,
          5,   16, 4472,  113,  103,   32,   15,   16, 5345,   19,  178,
         32])

In [8]:
len(x_train[0])

100

#### 4. Design and compile model

In [9]:
def create_model(dropout=0.5, n_dense=64):

    model = Sequential(name='model_dense')
    
    model.add(Embedding(10000, 64, input_length=100))
    model.add(Flatten())
    
    model.add(Dense(n_dense, activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

#### 5. Create model

In [10]:
model = KerasClassifier(build_fn=create_model, verbose=1)

#### 6. Define grid search parameters: `epochs` and `batch_size`

In [11]:
epochs = [4, 6]
batch_size = [64, 128]

param_grid = {'batch_size': batch_size, 'epochs': epochs}

In [12]:
param_grid

{'batch_size': [64, 128], 'epochs': [4, 6]}

In [13]:
grid_search = GridSearchCV(model, param_grid, cv=3, scoring = 'roc_auc')

#### 7. Perform grid search and print results

In [14]:
grid_search.fit(x_train, y_train)

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
16666/16666 [==============================] - 6s 349us/step - loss: 0.5079 - accuracy: 0.7365
Epoch 2/4
16666/16666 [==============================] - 5s 316us/step - loss: 0.1757 - accuracy: 0.9372
Epoch 3/4
16666/16666 [==============================] - 5s 316us/step - loss: 0.0286 - accuracy: 0.9951
Epoch 4/4
8334/8334 [==============================] - 0s 32us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
16667/16667 [==============================] - 5s 325us/step - loss: 0.5050 - accuracy: 0.7350
Epoch 2/4
16667/16667 [==============================] - 6s 340us/step - loss: 0.1725 - accuracy: 0.9356
Epoch 3/4
16667/16667 [==============================] - 5s 326us/step - loss: 0.0239 - accuracy: 0.9966
Epoch 4/4
8333/8333 [==============================] - 0s 34us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/6
16667/16667 [==============================] - 3s 184us/step - loss: 0.5774 - accuracy: 0.6834
Epoch 2/6
16667/16667 [==============================] - 3s 180us/step - loss: 0.2342 - accuracy: 0.9085
Epoch 3/6
16667/16667 [==============================] - 3s 180us/step - loss: 0.0565 - accuracy: 0.9883
Epoch 4/6
16667/16667 [==============================] - 3s 199us/step - loss: 0.0120 - accuracy: 0.9993
Epoch 5/6
16667/16667 [==============================] - 4s 234us/step - loss: 0.0045 - accuracy: 0.9999
Epoch 6/6
8333/8333 [==============================] - 0s 26us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
25000/25000 [==============================] - 5s 184us/step - loss: 0.5067 - accuracy: 0.7274
Epoch 2/4
25000/25000 [==============================] - 5s 182us/step - loss: 0.2070 - accuracy: 0.9233
Epoch 3/4
25000/25000 [==============================] - 5s 185us/step - loss: 0.0533 - accuracy: 0.9876
Epoch 4/4
25000/25000 [==============================] - 5s 185us/step - loss: 0.0099 - accuracy: 0.9991


GridSearchCV(cv=3, error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x0000021297ED7048>,
             iid='deprecated', n_jobs=None,
             param_grid={'batch_size': [64, 128], 'epochs': [4, 6]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [15]:
grid_search.best_params_

{'batch_size': 128, 'epochs': 4}

In [18]:
# create function to print results

import tabulate

def grid_search_results(cvres):
    
    columns = []
    table = []
    
    for key in param_grid.keys():

        columns.append(key)
    
    columns.append(grid_search.scoring)

    for mean_score, params in sorted(zip(cvres['mean_test_score'], cvres['params']), reverse=True):
        
        row = []

        for key in param_grid.keys():
            
            row.append(params[key])
        
        row.append(mean_score)
        table.append(row)
            
    print(tabulate.tabulate(table, headers=columns))

In [19]:
grid_search_results(grid_search.cv_results_)

  batch_size    epochs    roc_auc
------------  --------  ---------
         128         4   0.921958
         128         6   0.921423
          64         6   0.921402
          64         4   0.920898


#### 8. Create model

In [20]:
model = KerasClassifier(build_fn=create_model, epochs=4, batch_size=128, verbose=1)

#### 9. Define grid search parameters: `n_dense` and `dropout`

In [21]:
n_dense = [64, 128]
dropout = [0.2, 0.5]

param_grid = {'n_dense': n_dense, 'dropout': dropout}

In [22]:
param_grid

{'n_dense': [64, 128], 'dropout': [0.2, 0.5]}

In [23]:
grid_search = GridSearchCV(model, param_grid, cv=3, scoring = 'roc_auc')

#### 10. Perform grid search and print results

In [24]:
%%time

grid_search.fit(x_train, y_train)

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
16666/16666 [==============================] - 3s 196us/step - loss: 0.5455 - accuracy: 0.7014
Epoch 2/4
16666/16666 [==============================] - 3s 180us/step - loss: 0.1944 - accuracy: 0.9301
Epoch 3/4
16666/16666 [==============================] - 3s 168us/step - loss: 0.0329 - accuracy: 0.9956
Epoch 4/4
8334/8334 [==============================] - 0s 30us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
16667/16667 [==============================] - 3s 188us/step - loss: 0.5372 - accuracy: 0.7077
Epoch 2/4
16667/16667 [==============================] - 3s 181us/step - loss: 0.1915 - accuracy: 0.9297
Epoch 3/4
16667/16667 [==============================] - 3s 171us/step - loss: 0.0326 - accuracy: 0.9954
Epoch 4/4
8333/8333 [==============================] - 0s 26us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
16667/16667 [==============================] - 4s 236us/step - loss: 0.5512 - accuracy: 0.6965
Epoch 2/4
16667/16667 [==============================] - 4s 238us/step - loss: 0.1985 - accuracy: 0.9246
Epoch 3/4
16667/16667 [==============================] - 4s 232us/step - loss: 0.0322 - accuracy: 0.9951
Epoch 4/4
8333/8333 [==============================] - 0s 32us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
25000/25000 [==============================] - 5s 182us/step - loss: 0.4999 - accuracy: 0.7384
Epoch 2/4
25000/25000 [==============================] - 5s 184us/step - loss: 0.2044 - accuracy: 0.9232
Epoch 3/4
25000/25000 [==============================] - 4s 179us/step - loss: 0.0520 - accuracy: 0.9882
Epoch 4/4
25000/25000 [==============================] - 5s 183us/step - loss: 0.0103 - accuracy: 0.9993
Wall time: 3min 18s


GridSearchCV(cv=3, error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000002128BA2BD48>,
             iid='deprecated', n_jobs=None,
             param_grid={'dropout': [0.2, 0.5], 'n_dense': [64, 128]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [25]:
grid_search.best_params_

{'dropout': 0.5, 'n_dense': 64}

In [28]:
grid_search_results(grid_search.cv_results_)

  n_dense    dropout    roc_auc
---------  ---------  ---------
       64        0.5   0.921133
       64        0.2   0.919806
      128        0.5   0.919114
      128        0.2   0.917623


In [ ]:
# --> fazer gridsearch com os 4 parâmetros simultaneamente? (epochs, batch_size, n_dense e dropout)

# --> ver q demora mto tempo e optar por random search?

In [30]:
import multiprocessing

# number of cores
multiprocessing.cpu_count()

4

In [36]:
# n_jobs=-1 is the same as n_jobs=4 in this case 

grid_search = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, scoring = 'roc_auc')

In [37]:
%%time

grid_search.fit(x_train, y_train)

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
25000/25000 [==============================] - 5s 207us/step - loss: 0.5005 - accuracy: 0.7360
Epoch 2/4
25000/25000 [==============================] - 5s 191us/step - loss: 0.2015 - accuracy: 0.9257
Epoch 3/4
25000/25000 [==============================] - 5s 189us/step - loss: 0.0498 - accuracy: 0.9882
Epoch 4/4
25000/25000 [==============================] - 5s 189us/step - loss: 0.0091 - accuracy: 0.9994
Wall time: 3min 4s


GridSearchCV(cv=3, error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000002128BA2BD48>,
             iid='deprecated', n_jobs=-1,
             param_grid={'dropout': [0.2, 0.5], 'n_dense': [64, 128]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [ ]:
# full progress output only on the terminal window and not on the notebook itself
# This may also interfere with the main neural network training process... Let's not use it...

#### 11. Create model

In [42]:
model = KerasClassifier(build_fn=create_model, verbose=1)

#### 12. Define grid search parameters: `epochs`, `batch_size`, `n_dense` and `dropout`

In [43]:
epochs = [4, 6]
batch_size = [64, 128]

n_dense = [64, 128]
dropout = [0.2, 0.5]

param_grid = {'epochs': epochs, 'batch_size': batch_size, 'n_dense': n_dense, 'dropout': dropout}

In [44]:
param_grid

{'epochs': [4, 6],
 'batch_size': [64, 128],
 'n_dense': [64, 128],
 'dropout': [0.2, 0.5]}

In [45]:
grid_search = GridSearchCV(model, param_grid, cv=3, scoring = 'roc_auc')

#### 13. Perform grid search and print results

In [46]:
%%time

grid_search.fit(x_train, y_train)

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
16666/16666 [==============================] - 6s 375us/step - loss: 0.4827 - accuracy: 0.7482
Epoch 2/4
16666/16666 [==============================] - 6s 340us/step - loss: 0.1347 - accuracy: 0.9560
Epoch 3/4
16666/16666 [==============================] - 5s 325us/step - loss: 0.0146 - accuracy: 0.9984
Epoch 4/4
8334/8334 [==============================] - 0s 34us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
16667/16667 [==============================] - 5s 311us/step - loss: 0.4880 - accuracy: 0.7467
Epoch 2/4
16667/16667 [==============================] - 5s 309us/step - loss: 0.1400 - accuracy: 0.9504
Epoch 3/4
16667/16667 [==============================] - 5s 310us/step - loss: 0.0160 - accuracy: 0.9977
Epoch 4/4
8333/8333 [==============================] - 0s 36us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/6
16667/16667 [==============================] - 4s 227us/step - loss: 0.5477 - accuracy: 0.6993
Epoch 2/6
16667/16667 [==============================] - 4s 226us/step - loss: 0.1999 - accuracy: 0.9238
Epoch 3/6
16667/16667 [==============================] - 4s 227us/step - loss: 0.0320 - accuracy: 0.9946
Epoch 4/6
16667/16667 [==============================] - 4s 229us/step - loss: 0.0056 - accuracy: 0.9998
Epoch 5/6
16667/16667 [==============================] - 4s 229us/step - loss: 0.0023 - accuracy: 1.0000
Epoch 6/6
8333/8333 [==============================] - 0s 36us/step


C:\Users\joaon\Anaconda3\envs\DL-illustrated\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/6
25000/25000 [==============================] - 5s 182us/step - loss: 0.5025 - accuracy: 0.7370
Epoch 2/6
25000/25000 [==============================] - 4s 179us/step - loss: 0.2001 - accuracy: 0.9261
Epoch 3/6
25000/25000 [==============================] - 4s 180us/step - loss: 0.0477 - accuracy: 0.9892
Epoch 4/6
25000/25000 [==============================] - 5s 182us/step - loss: 0.0095 - accuracy: 0.9994
Epoch 5/6
25000/25000 [==============================] - 5s 219us/step - loss: 0.0030 - accuracy: 1.0000
Epoch 6/6
25000/25000 [==============================] - 5s 210us/step - loss: 0.0017 - accuracy: 1.0000
Wall time: 20min 39s


GridSearchCV(cv=3, error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000002128BA3ECC8>,
             iid='deprecated', n_jobs=None,
             param_grid={'batch_size': [64, 128], 'dropout': [0.2, 0.5],
                         'epochs': [4, 6], 'n_dense': [64, 128]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [47]:
grid_search.best_params_

{'batch_size': 128, 'dropout': 0.5, 'epochs': 6, 'n_dense': 64}

In [48]:
grid_search_results(grid_search.cv_results_)

  epochs    batch_size    n_dense    dropout    roc_auc
--------  ------------  ---------  ---------  ---------
       6           128         64        0.5   0.922937
       6            64         64        0.5   0.921665
       6           128         64        0.2   0.921069
       4           128         64        0.5   0.92064
       6            64         64        0.2   0.920537
       6            64        128        0.5   0.920433
       6           128        128        0.5   0.920358
       4           128         64        0.2   0.920281
       4           128        128        0.5   0.919852
       4            64         64        0.5   0.919536
       6           128        128        0.2   0.919005
       4           128        128        0.2   0.91807
       4            64         64        0.2   0.917793
       4            64        128        0.5   0.917669
       6            64        128        0.2   0.916376
       4            64        128        0.2   0.9

In [68]:
df_cv_results = pd.DataFrame(grid_search.cv_results_['params'], columns = ['epochs', 'batch_size', 'n_dense', 'dropout'])

In [69]:
df_cv_results['roc_auc'] = grid_search.cv_results_['mean_test_score']

In [79]:
df_cv_results.sort_values(by=['roc_auc'], ascending=False)

,epochs,batch_size,n_dense,dropout,roc_auc
14,6,128,64,0.5,0.922937
6,6,64,64,0.5,0.921665
10,6,128,64,0.2,0.921069
12,4,128,64,0.5,0.920640
2,6,64,64,0.2,0.920537
7,6,64,128,0.5,0.920433
15,6,128,128,0.5,0.920358
8,4,128,64,0.2,0.920281
13,4,128,128,0.5,0.919852
4,4,64,64,0.5,0.919536


In [83]:
print(df_cv_results.sort_values(by=['roc_auc'], ascending=False).to_string(index=False))

 epochs  batch_size  n_dense  dropout   roc_auc
      6         128       64      0.5  0.922937
      6          64       64      0.5  0.921665
      6         128       64      0.2  0.921069
      4         128       64      0.5  0.920640
      6          64       64      0.2  0.920537
      6          64      128      0.5  0.920433
      6         128      128      0.5  0.920358
      4         128       64      0.2  0.920281
      4         128      128      0.5  0.919852
      4          64       64      0.5  0.919536
      6         128      128      0.2  0.919005
      4         128      128      0.2  0.918070
      4          64       64      0.2  0.917793
      4          64      128      0.5  0.917669
      6          64      128      0.2  0.916376
      4          64      128      0.2  0.916114


In [85]:
from IPython.display import display, HTML
display(HTML(df_cv_results.sort_values(by=['roc_auc'], ascending=False).to_html(index=False)))

epochs,batch_size,n_dense,dropout,roc_auc
6,128,64,0.5,0.922937
6,64,64,0.5,0.921665
6,128,64,0.2,0.921069
4,128,64,0.5,0.920640
6,64,64,0.2,0.920537
6,64,128,0.5,0.920433
6,128,128,0.5,0.920358
4,128,64,0.2,0.920281
4,128,128,0.5,0.919852
4,64,64,0.5,0.919536


In [98]:
# why the change in the number of decimal places in the dropout column?!

df_cv_results.sort_values(by=['roc_auc'], ascending=False).style.hide_index()

epochs,batch_size,n_dense,dropout,roc_auc
6,128,64,0.500000,0.922937
6,64,64,0.500000,0.921665
6,128,64,0.200000,0.921069
4,128,64,0.500000,0.920640
6,64,64,0.200000,0.920537
6,64,128,0.500000,0.920433
6,128,128,0.500000,0.920358
4,128,64,0.200000,0.920281
4,128,128,0.500000,0.919852
4,64,64,0.500000,0.919536


In [104]:
format_dict = {'dropout': '{:.1f}'}
df_cv_results.sort_values(by=['roc_auc'], ascending=False).style.format(format_dict).hide_index()

epochs,batch_size,n_dense,dropout,roc_auc
6,128,64,0.5,0.922937
6,64,64,0.5,0.921665
6,128,64,0.2,0.921069
4,128,64,0.5,0.920640
6,64,64,0.2,0.920537
6,64,128,0.5,0.920433
6,128,128,0.5,0.920358
4,128,64,0.2,0.920281
4,128,128,0.5,0.919852
4,64,64,0.5,0.919536


In [108]:
grid_search.best_score_

0.922937381396063

As we've seen, even for a relatively simple model and only two options for each of the four hyperparameters we decided to tune (and with a low cross-validation value of 3), our last grid search took more than 20 minutes to complete.

One alterative is to perform a **random search** instead. This will be our **step 2**.

### Step 2